In [33]:
%pip install --upgrade DBcm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\fondo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [34]:
import DBcm

In [35]:
creds = {
    "host": "localhost",
    "database": "BlackjackDB",
    "user": "root",
    "password": "setu2023",
}

In [36]:
def random_gamer():
    with DBcm.UseDatabase(creds) as db:
        SQL = """
            select gamertag from players
        """
        db.execute(SQL)
        tags = db.fetchall()
        tags = [ row[0] for row in tags ]
        who = random.choice(tags)
    return who

In [37]:
import random
import DBcm

def get_current_attempts(gamertag):
    with DBcm.UseDatabase(creds) as db:
        SQL = """
            SELECT COUNT(*) as attempts
            FROM games
            WHERE gamertag = %s
        """
        db.execute(SQL, (gamertag,))
        result = db.fetchone()
        return result[0] if result else 0

def get_current_wins(gamertag):
    with DBcm.UseDatabase(creds) as db:
        SQL = """
            SELECT COUNT(*) as wins
            FROM games
            WHERE gamertag = %s AND outcome = 'Win'
        """
        db.execute(SQL, (gamertag,))
        result = db.fetchone()
        return result[0] if result else 0

for _ in range(500):
    SQL = """
        INSERT INTO games
        (gamertag, outcome, attempts, wins)
        VALUES (%s, %s, %s, %s)
    """
    gamer = random_gamer()
    outcome = random.choice(["Win", "Loss"])
    attempts = get_current_attempts(gamer) + 1
    wins = get_current_wins(gamer) + (outcome == "Win")

    with DBcm.UseDatabase(creds) as db:
            db.execute(SQL, (gamer, outcome, attempts, wins))
